In [364]:
import requests
import re
import pandas as pd
import io
import html5lib
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
import platform
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

Any urls we may need

In [365]:
_BASE_URL = 'https://verd.lyfjastofnun.is/'
_URL = 'https://verd.lyfjastofnun.is/?pageid=10'

# **Extract**

Fetch the urls of all excel files, do not get xlsx files that were uploaded due to them being a security issue.

https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file

In [366]:
r = requests.get(_URL)
soup = bs(r.text)
#xlsx
_links = soup.find_all('a', {'href': re.compile(r'verdskra.*\.xls$')})

Iterate through all excel files and add to our dataframe for processing.

Skip the first file as it simply explains the columns of the rest of the files.

In [367]:
appended_data = []
for index, link in enumerate(_links[1:]):
    price_link = link.get("href")
    file_name = re.search('gogn/(.*).xls',price_link).group(1)
    response = requests.get(_BASE_URL + price_link).content#
    inc_data = pd.read_excel(response).assign(file=file_name)
    appended_data.append(inc_data)

data = pd.concat(appended_data)
data.head(10)

,Norrænt vörunúmer,Lyfjaheitisnúmer,Heiti lyfs,Form lyfs,Styrkur,Styrkeining,Magn,Magneining,Fj. Pakkninga,Pakkning,...,"Skammtur að hámarki fyrir fjölda daga (100, 30)",Heildsöluverð,Smásöluverð,Meðferð,Hmagn,"Á undanþágulista (1, 0)",DDD,Unnamed: 1,Afskáningardags,Unnamed: 36
0,379875.0,N05BA0105,Stesolid,eþ-lausn,5,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,379883.0,N05BA0105,Stesolid,eþ-lausn,10,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,43729.0,N05CD0503,Halcion,töflur,"0,125",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,48199.0,A03CA0201,Librax,töflur,NaN,NaN,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,52286.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,52297.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,52309.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,52320.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,52331.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
9,70203.0,N05CF0107,Imomed,filmhtfl,"7,5",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


# **Transform**

Throughout the years the columns have different names, e.g.

```
   form   Styrkur Styrk form lyfs
0   töflur 1.0    NaN   NaN   
1   töflur 5.0    NaN   NaN   
2   NaN    NaN    1.0   töflur   
3   NaN    NaN    1.0   töflur   
4   NaN    NaN    1.0   töflur
```

Time to clean the data as best we can. Lets merge all the data and rename some columns to something clearer.

In [368]:
data['Norrænt vörunúmer'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
data['Form'] = data['Form lyfs'].combine_first(data['Form'])
data['Heiti'] = data['Heiti lyfs'].combine_first(data['Heiti'])
data['Fj. Pakkninga'] = data['Fj. Pakkninga'].combine_first(data['Fjöldi pakkninga'])
data['Fj. Pakkninga'] = data['Fj. Pakkninga'].combine_first(data['Fj í pakkningu'])
data['Tannlæknir ávísar'] = data['Tannlæknir ávísar'].combine_first(data['Tannlæknir ávísar (1, 0)'])
data['Dýralæknir ávísar'] = data['Dýralæknir ávísar'].combine_first(data['Dýralæknir ávísar  (1, 0)'])
data['Sérfræðingur ávísar'] = data['Sérfræðingur ávísar'].combine_first(data['Sérfræðingur ávísar  (1, 0)'])
data['Markaðsleyfishafi kóði'] = data['Markaðsleyfishafi kóði'].combine_first(data['Markaðsleyfishafi-Kódi'])
data['Sjúkrahúslyf'] = data['Sjúkrahúslyf'].combine_first(data['Sjúkrahúslyf  (1, 0)'])
data['Á undanþágulista'] = data['Á undanþágulista'].combine_first(data['Á undanþágulista (1, 0)'])
data['Greiðsluþátttökuverð'] = data['Greiðsluþátttökuverð'].combine_first(data['Greiðsluþátttökusmásöluverð'])
data['Skammtur, hámark daga'] = data['Skammtur, hámark daga'].combine_first(data['Skammtur að hámarki fyrir fjölda daga  (100,  30)'])
data['DDD'] = data['DDD á pakkningu'].combine_first(data['DDD'])
data['Styrkur'] = data['Styrkur'].combine_first(data['Tölugildi styrks'])
data['Styrkeining'] = data['Styrkeining'].combine_first(data['Einingar styrks'])
data['Styrkeining'] = data['Styrkeining'].combine_first(data['Einstyrks'])
data['Styrkeining'] = data['Styrkeining'].combine_first(data['Eining styrks'])
data['Magn'] = data['Magn'].combine_first(data['Tölugildi magns'])
data['Magneining'] = data['Magneining'].combine_first(data['Einingar magns'])
data['Magneining'] = data['Magneining'].combine_first(data['Einmagns'])
data['Heildarmagn'] = data['Heildarmagn'].combine_first(data['Hmagn'])
data['Meðferð'] = data['Meðferð'].combine_first(data['Leiðbeiningar um meðferð'])
data['Geymsluþol'] = data['Geymsluþol'].combine_first(data['Geymsluþol  (F,H,K,S,A)'])
data['Geymsluþol'] = data['Geymsluþol'].combine_first(data['Geymslutími (0-5, G>= 5ár)'])
data['Viðmiðunargjaldmiðill'] = data['Viðmiðunargjaldmiðill'].combine_first(data['Viðmiðunargjald'])
data['Ákvörðun LGN'] = data['Ákvörðun LGN um greiðsluþátttöku'].combine_first(data['Ákvörðun LGN'])
data['Greiðsluhlutur SÍ'] = data['Greiðsluhlutur SÍ'].combine_first(data['Greiðsluhlutur TR'])
data['Geðlyf á geðdeild'] = data['Geðlyf á geðdeild'].combine_first(data['Geðlyf á geðdeild  (1, 0)'])
data['Eftirritunarskylt'] = data['Eftirritunarskylt'].combine_first(data['Eftirritunarskylt  (1, 0)'])
data['Skráningardagsetning'] = data['Skráningardagsetning'].combine_first(data['Skráningardags'])
data['Afskráningardagsetning'] = data['Afskráningardagsetning'].combine_first(data['Afskáningardags'])
data['Afskráningardagsetning'] = data['Afskráningardagsetning'].combine_first(data['Afskráning'])

## Drop our merged columns
data = data.drop(columns=[
'Norrænt vörunúmer',
'Form lyfs',
'Fjöldi pakkninga','Fj í pakkningu',
'Tannlæknir ávísar (1, 0)',
'Heiti lyfs',
'Markaðsleyfishafi kóði',
'Dýralæknir ávísar  (1, 0)',
'Sérfræðingur ávísar  (1, 0)',
'Sjúkrahúslyf  (1, 0)',
'Á undanþágulista (1, 0)',
'Greiðsluþátttökusmásöluverð',
'Skammtur að hámarki fyrir fjölda daga  (100,  30)',
'DDD á pakkningu',
'Styrkeining','Einstyrks','Einingar styrks','Eining styrks','Tölugildi styrks',
'Magneining','Einmagns','Einingar magns','Tölugildi magns',
'Heildarmagn',
'Meðferð',
'Unnamed: 0','Unnamed: 1','Unnamed: 36',
'Viðmiðunargjald',
'Ákvörðun LGN um greiðsluþátttöku',
'Greiðsluhlutur TR',
'Geðlyf á geðdeild  (1, 0)',
'Eftirritunarskylt  (1, 0)',
'Skráningardags',
'Geymsluþol  (F,H,K,S,A)',
'Afskráning','Afskáningardags'])


In [369]:
data = data.rename(columns={
'Afgreiðslumáti  (R,L,U)':'Afgreiðslumáti',
'Geymslutími (0-5, G>= 5ár)':'Geymslutími',
'Greiðsluhlutur Tr  (0,*,B,E)':'Greiðsluhlutur Tr'
})

In [363]:
data['Hámarks heildsöluverð'] = data['Hámarks heildsöluverð'].str.replace(r'\.', '')
data['Hámarks heildsöluverð'] = data[['Hámarks heildsöluverð']].apply(pd.to_numeric)
data['Hámarks smásöluverð'] = data['Hámarks smásöluverð'].str.replace(r'\.', '')
data['Hámarks smásöluverð'] = data[['Hámarks smásöluverð']].apply(pd.to_numeric)
data['Lyfjaframleiðenda heildsöluverð'] = data['Lyfjaframleiðenda heildsöluverð'].str.replace(r'\.', '')
data['Lyfjaframleiðenda heildsöluverð'] = data[['Lyfjaframleiðenda heildsöluverð']].apply(pd.to_numeric)

KeyError: "None of [Index(['Skammtur að hámarki fyrir fjölda daga  (100,  30)'], dtype='object')] are in the [columns]"

In [347]:
data.columns

Index(['Lyfjaheitisnúmer', 'Styrkur', 'Magn', 'Fj. Pakkninga', 'Pakkning',
       'Athugasemdir', 'ATC-flokkur', 'Skráningardagsetning',
       'Afskráningardagsetning', 'Afgreiðslumáti', 'Skömmtunarmöguleikar ',
       'Öryggiskröfur LST', 'Sérfræðingur ávísar', 'Greiðsluhlutur SÍ',
       'Sjúkrahúslyf', 'Dýralæknir ávísar', 'Leyfisskyld lyf ',
       'Tannlæknir ávísar', 'Skammtur, hámark daga', 'Viðmiðunargjaldmiðill',
       'Hámarks heildsöluverð', 'Hámarks smásöluverð', 'Álagningarflokkur',
       'Viðmiðunarverðflokkur', 'Viðmiðunarverð', 'Staða lyfs ',
       'Á undanþágulista', 'Umboðsaðili', 'Markaðsleyfishafi',
       'Heiti virka innihaldsefnis', 'Lyfjaframleiðenda heildsöluverð',
       'Umboðsmanna heildsöluverð', 'Heildsölu heildsöluverð',
       'Innkaupsverð apóteka', 'Greiðsluþátttökuverð',
       'Fellt úr lyfjaverðskrá vegna birgðaskorts', 'Heildsali', 'H merking',
       'Ávana og fíknilyf', 'Ávana og fíknilyf magn',
       'Ávana og fíknilyf eining', 'Hjúkr-/ljós

There's no load step since this is a simple notebook so let's just see some data

### For each file released throughout the years, what was the most expensive wholesale drug?

In [348]:
#data.groupby(['file','Heiti'])['Hámarks heildsöluverð'].unique()

test = data[['Hámarks heildsöluverð']].apply(pd.to_numeric)

In [349]:
n = 100
idx = pd.date_range(start=datetime(2016, 1, 1, 10), freq='10Min', periods=n)
data = pd.DataFrame(data={'price': ),
                          'volume': np.random.randint(low=100, high=10000, size=n)}, index=idx)

fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(15,8))

ax[0].plot(data.index, data.price)
ax[1].bar(data.index, data.volume, width=1/(5*len(data.index)))

xfmt = mpl.dates.DateFormatter('%H:%M')
ax[1].xaxis.set_major_locator(mpl.dates.HourLocator(interval=3))
ax[1].xaxis.set_major_formatter(xfmt)

ax[1].xaxis.set_minor_locator(mpl.dates.HourLocator(interval=1))
ax[1].xaxis.set_minor_formatter(xfmt)

ax[1].get_xaxis().set_tick_params(which='major', pad=25)

fig.autofmt_xdate()
plt.show()

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (<ipython-input-349-88420184a3bb>, line 3)